<a href="https://colab.research.google.com/github/Chandana909/ML-Customer-Churn-Predictor-Project/blob/main/ChurnPredictorProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, r2_score
import datetime
from dateutil.relativedelta import relativedelta

# Global variables
df = None
models = {}
feature_importance = {}

# ====================== PROFESSIONAL UI SETUP ======================
CSS = """
:root {
    --primary: #1a4b8c;
    --secondary: #3a7bd5;
    --accent: #FF7F50;
    --background: #e6f0fa;
    --card: #FFFFFF;
    --text: #333333;
    --font-heading: 'Montserrat', sans-serif;
}
@import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@600;700&display=swap');
body, html {
    background-color: var(--background) !important;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    margin: 0;
    padding: 0;
    height: 100%;
}
.dashboard-header {
    background: linear-gradient(135deg, var(--primary), var(--secondary));
    color: white !important;
    padding: 2rem;
    margin-bottom: 1.5rem;
    text-align: center;
    box-shadow: 0 4px 12px rgba(0,0,0,0.1);
    font-family: var(--font-heading);
    border-bottom: 4px solid var(--accent);
}
.dashboard-header h1 {
    font-size: 2.5rem;
    margin: 0;
    letter-spacing: 1px;
    text-shadow: 1px 1px 3px rgba(0,0,0,0.2);
}
.dashboard-header p {
    font-size: 1.2rem;
    margin: 0.5rem 0 0;
    opacity: 0.9;
}
.card {
    background: var(--card);
    border-radius: 10px;
    padding: 1.5rem;
    margin-bottom: 1.5rem;
    box-shadow: 0 4px 12px rgba(0,0,0,0.08);
    border: none;
    transition: transform 0.2s, box-shadow 0.2s;
}
.card:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 16px rgba(0,0,0,0.12);
}
.alert-success {
    color: #28A745;
    background-color: #E8F5E9;
    border-left: 4px solid #28A745;
    padding: 1rem;
    border-radius: 4px;
}
.alert-error {
    color: #DC3545;
    background-color: #F8E8E8;
    border-left: 4px solid #DC3545;
    padding: 1rem;
    border-radius: 4px;
}
.gradio-container {
    max-width: 100% !important;
    padding: 20px !important;
    background-color: var(--background) !important;
    min-height: 100vh;
}
.section-title {
    font-size: 1.3rem;
    font-weight: 600;
    color: var(--primary) !important;
    margin-bottom: 1rem;
    border-bottom: 2px solid var(--secondary);
    padding-bottom: 0.5rem;
}
.gradio-plot {
    border-radius: 10px !important;
    background: white !important;
    padding: 1rem !important;
    border: none !important;
    box-shadow: 0 2px 8px rgba(0,0,0,0.05) !important;
}
.tab-button {
    font-weight: 500 !important;
    padding: 0.8rem 1.5rem !important;
}
.tab-button.selected {
    background: var(--primary) !important;
    color: white !important;
}
.consumer-profile {
    background: linear-gradient(135deg, #f5f7fa 0%, #e4e8eb 100%);
    padding: 1.5rem;
    border-radius: 10px;
    margin-bottom: 1.5rem;
}
.consumer-profile h4 {
    color: var(--primary);
    margin-bottom: 1rem;
}
"""

# ====================== DATA PROCESSING ======================
def preprocess_data(file_obj):
    global df, models, feature_importance

    try:
        if file_obj is None:
            return "<div class='alert-error'>❌ Please upload a file first</div>", None

        # Reset previous data and models
        df = None
        models = {}
        feature_importance = {}

        df = pd.read_csv(file_obj.name)

        # Check required columns (removed Date from required columns)
        required_cols = ['CustomerID', 'Age', 'Annual_Income', 'Spending_Score', 'Purchase_History']
        if not all(col in df.columns for col in required_cols):
            missing = [col for col in required_cols if col not in df.columns]
            return f"<div class='alert-error'>❌ Missing columns: {', '.join(missing)}</div>", None

        # Add Date column if not present (using current date)
        if 'Date' not in df.columns:
            df['Date'] = datetime.datetime.now().strftime('%Y-%m-%d')

        # Convert currency to ₹ (Indian Rupees)
        df['Annual_Income_INR'] = df['Annual_Income'] * 75

        # Feature engineering
        df['Date'] = pd.to_datetime(df['Date'])
        df['Days_Since_First_Purchase'] = (df['Date'] - df['Date'].min()).dt.days
        df['Purchase_Frequency'] = df['Purchase_History'] / df['Days_Since_First_Purchase'].replace(0, 1)
        df['Customer_Lifetime_Value'] = (df['Annual_Income_INR'] * 0.1) + (df['Purchase_History'] * 500)

        # Generate summary stats
        stats = {
            "Total Customers": f"{len(df):,}",
            "Average Age": f"{df['Age'].mean():.1f} years",
            "Average Income (₹)": f"₹{df['Annual_Income_INR'].mean():,.0f}",
            "Avg. Spending Score": f"{df['Spending_Score'].mean():.1f}/100",
            "Total Purchases": f"{df['Purchase_History'].sum():,}",
            "Data Coverage": f"{df['Date'].min().strftime('%b %Y')} to {df['Date'].max().strftime('%b %Y')}",
            "Avg. Customer Value": f"₹{df['Customer_Lifetime_Value'].mean():,.0f}"
        }

        stats_html = "<div class='card'><h4 class='section-title'>📊 Dataset Summary</h4><ul style='columns: 2;'>"
        for k, v in stats.items():
            stats_html += f"<li style='margin-bottom: 0.5rem;'><strong>{k}:</strong> {v}</li>"
        stats_html += "</ul></div>"

        return "<div class='alert-success'>✅ Data processed successfully! ML can now analyze patterns.</div>", stats_html
    except Exception as e:
        return f"<div class='alert-error'>❌ Error: {str(e)}</div>", None

# ====================== MODEL TRAINING ======================
def train_models():
    global df, models, feature_importance

    if df is None:
        return "<div class='alert-error'>❌ No data available. Please upload and process data first.</div>", None, None

    try:
        # Prepare features with ML explanations
        features = ['Age', 'Annual_Income', 'Spending_Score', 'Purchase_History',
                   'Days_Since_First_Purchase', 'Purchase_Frequency']

        X = df[features]
        y_churn = (df['Spending_Score'] < 40).astype(int)  # Churn = low spending score
        y_sales = df['Purchase_History']
        y_clv = df['Customer_Lifetime_Value']

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y_churn, test_size=0.3, random_state=42)

        # Train models with ML explanations
        churn_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
        sales_model = RandomForestRegressor(n_estimators=100, random_state=42)
        clv_model = RandomForestRegressor(n_estimators=100, random_state=42)

        churn_model.fit(X_train, y_train)
        sales_model.fit(X_train, y_sales[X_train.index])
        clv_model.fit(X_train, y_clv[X_train.index])

        # Store models and metrics
        models = {
            'churn': churn_model,
            'sales': sales_model,
            'clv': clv_model,
            'metrics': {
                'churn_accuracy': accuracy_score(y_test, churn_model.predict(X_test)),
                'churn_auc': roc_auc_score(y_test, churn_model.predict_proba(X_test)[:,1]),
                'sales_r2': r2_score(y_sales[X_test.index], sales_model.predict(X_test)),
                'clv_r2': r2_score(y_clv[X_test.index], clv_model.predict(X_test))
            }
        }

        # Feature importance only
        feature_importance = {
            'churn': churn_model.feature_importances_,
            'sales': sales_model.feature_importances_,
            'clv': clv_model.feature_importances_
        }

        # Model cards with ML explanations
        model_cards = f"""
        <div class='card'>
            <h4 class='section-title'>🤖 Model Performance</h4>
            <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 1rem;">
                <div>
                    <h5>Churn Model</h5>
                    <p><i>ML predicts customers likely to stop purchasing based on spending patterns</i></p>
                    <p><strong>Accuracy:</strong> {models['metrics']['churn_accuracy']:.2f}</p>
                    <p><strong>AUC:</strong> {models['metrics']['churn_auc']:.2f}</p>
                </div>
                <div>
                    <h5>Sales Model</h5>
                    <p><i>ML forecasts future purchase volumes using customer behavior patterns</i></p>
                    <p><strong>R²:</strong> {models['metrics']['sales_r2']:.2f}</p>
                </div>
                <div>
                    <h5>CLV Model</h5>
                    <p><i>ML estimates long-term customer value from income and purchase history</i></p>
                    <p><strong>R²:</strong> {models['metrics']['clv_r2']:.2f}</p>
                </div>
            </div>
        </div>
        """

        # Generate feature importance plot with ML explanation
        feature_fig = plot_feature_importance()

        return "<div class='alert-success'>✅ Models trained successfully!</div>", model_cards, feature_fig
    except Exception as e:
        return f"<div class='alert-error'>❌ Training error: {str(e)}</div>", None, None

def plot_feature_importance():
    global feature_importance, df
    if not feature_importance or df is None:
        return None

    try:
        features = ['Age', 'Income', 'Spending', 'Purchases', 'Days', 'Frequency']

        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=features,
            y=feature_importance['churn'],
            name='Churn Importance',
            marker_color='#1a4b8c',
            hovertemplate='<b>%{x}</b><br>Importance: %{y:.2f}<extra></extra>'
        ))
        fig.add_trace(go.Bar(
            x=features,
            y=feature_importance['sales'],
            name='Sales Importance',
            marker_color='#3a7bd5',
            hovertemplate='<b>%{x}</b><br>Importance: %{y:.2f}<extra></extra>'
        ))
        fig.add_trace(go.Bar(
            x=features,
            y=feature_importance['clv'],
            name='CLV Importance',
            marker_color='#5ca0ff',
            hovertemplate='<b>%{x}</b><br>Importance: %{y:.2f}<extra></extra>'
        ))
        fig.update_layout(
            title='<b>Feature Importance</b><br><i>ML quantifies how much each factor influences predictions</i>',
            barmode='group',
            plot_bgcolor='white',
            paper_bgcolor='white',
            height=400,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        return fig
    except Exception as e:
        print(f"Feature importance error: {str(e)}")
        return None

# ====================== ANALYTICS ======================
def generate_sales_analytics():
    global df
    if df is None:
        return "<div class='alert-error'>Please upload and process data first</div>"

    try:
        # Ensure Date column exists
        if 'Date' not in df.columns:
            df['Date'] = datetime.datetime.now()

        # Sales statistics
        monthly_sales = df.set_index('Date')['Purchase_History'].resample('M').sum()
        total_sales = monthly_sales.sum()
        avg_monthly = monthly_sales.mean()
        growth_rate = (monthly_sales.iloc[-1] - monthly_sales.iloc[0]) / monthly_sales.iloc[0] * 100 if len(monthly_sales) > 1 else 0

        stats = {
            "Total Sales": f"{total_sales:,.0f}",
            "Average Monthly Sales": f"{avg_monthly:,.0f}",
            "Growth Rate": f"{growth_rate:.1f}%",
            "Best Month": f"{monthly_sales.idxmax().strftime('%b %Y')} ({monthly_sales.max():,.0f})" if len(monthly_sales) > 0 else "N/A",
            "Worst Month": f"{monthly_sales.idxmin().strftime('%b %Y')} ({monthly_sales.min():,.0f})" if len(monthly_sales) > 0 else "N/A"
        }

        stats_html = "<div class='card'><h4 class='section-title'>📊 Sales Statistics</h4><ul style='columns: 2;'>"
        for k, v in stats.items():
            stats_html += f"<li style='margin-bottom: 0.5rem;'><strong>{k}:</strong> {v}</li>"
        stats_html += "</ul></div>"

        # Monthly trend
        monthly_df = monthly_sales.reset_index()
        monthly_df['Rolling_Avg'] = monthly_df['Purchase_History'].rolling(3, min_periods=1).mean()

        trend_fig = px.line(
            monthly_df,
            x='Date',
            y=['Purchase_History', 'Rolling_Avg'],
            title='<b>Monthly Sales Trend</b><br><i>ML identifies patterns in customer purchasing behavior</i>',
            color_discrete_sequence=['#1a4b8c', '#3a7bd5'],
            height=350
        )
        trend_fig.update_layout(
            plot_bgcolor='white',
            paper_bgcolor='white',
            legend_title="Metric"
        )

        # Customer segments
        segment_fig = px.box(
            df,
            x=pd.cut(df['Spending_Score'], bins=[0, 40, 70, 100], labels=['Low', 'Medium', 'High']),
            y='Purchase_History',
            title='<b>Purchases by Spending Segment</b><br><i>ML analyzes how spending behavior affects purchase volume</i>',
            color_discrete_sequence=['#1a4b8c'],
            height=300
        )
        segment_fig.update_layout(
            xaxis_title="Spending Score Segment",
            yaxis_title="Purchases",
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

        analytics_html = f"""
        <div class='card'>
            <h4 class='section-title'>📈 Sales Analytics</h4>
            {stats_html}
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 1rem; margin-top: 1rem;">
                <div>{trend_fig.to_html(full_html=False, include_plotlyjs='cdn')}</div>
                <div>{segment_fig.to_html(full_html=False, include_plotlyjs='cdn')}</div>
            </div>
        </div>
        """

        return analytics_html
    except Exception as e:
        return f"<div class='alert-error'>Sales analytics error: {str(e)}</div>"

# ====================== PLANNING ======================
def generate_production_plan():
    global df
    if df is None:
        return "<div class='alert-error'>Please upload and process data first</div>"

    try:
        # Ensure Date column exists
        if 'Date' not in df.columns:
            df['Date'] = datetime.datetime.now()

        monthly_sales = df.set_index('Date')['Purchase_History'].resample('M').sum()

        # ML-enhanced seasonal factors
        seasonal_factor = {
            1: 1.3, 2: 1.1, 3: 1.0, 4: 0.9, 5: 0.95, 6: 0.85,
            7: 0.8, 8: 0.9, 9: 1.0, 10: 1.1, 11: 1.4, 12: 1.5
        }

        # Generate 12-month plan
        current_date = datetime.datetime.now()
        months = [current_date + relativedelta(months=i) for i in range(12)]
        plan = []

        for month in months:
            month_name = month.strftime('%b %Y')
            factor = seasonal_factor.get(month.month, 1.0)
            predicted_sales = monthly_sales.mean() * factor if len(monthly_sales) > 0 else 0

            if month.month in [11, 12, 1]:
                plan.append({
                    "month": month_name,
                    "action": "Increase +30%",
                    "reason": "Peak season (ML predicts ↑ sales)",
                    "prediction": f"{predicted_sales:,.0f} (↑{factor*100-100:.0f}%)"
                })
            elif month.month in [6, 7, 8]:
                plan.append({
                    "month": month_name,
                    "action": "Reduce -20%",
                    "reason": "Slow season (ML predicts ↓ sales)",
                    "prediction": f"{predicted_sales:,.0f} (↓{100-factor*100:.0f}%)"
                })
            else:
                plan.append({
                    "month": month_name,
                    "action": "Standard",
                    "reason": "Typical sales period",
                    "prediction": f"{predicted_sales:,.0f}"
                })

        # Create sales trend visualization
        sales_fig = px.line(
            monthly_sales.reset_index(),
            x='Date',
            y='Purchase_History',
            title='<b>Historical Sales</b><br><i>ML detects trends for inventory optimization</i>',
            color_discrete_sequence=['#1a4b8c'],
            height=300
        )
        sales_fig.update_layout(
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

        # Create plan table
        plan_html = "<table style='width: 100%; border-collapse: collapse;'>"
        plan_html += """
        <tr style='background-color: #1a4b8c; color: white;'>
            <th style='padding: 10px; text-align: left;'>Month</th>
            <th style='padding: 10px; text-align: left;'>Action</th>
            <th style='padding: 10px; text-align: left;'>ML Insight</th>
            <th style='padding: 10px; text-align: right;'>Forecast</th>
        </tr>
        """
        for i, item in enumerate(plan):
            bg_color = '#f0f8ff' if i % 2 == 0 else 'white'
            icon = "📈" if "Increase" in item["action"] else "📉" if "Reduce" in item["action"] else "⚖️"
            plan_html += f"""
            <tr style='background-color: {bg_color};'>
                <td style='padding: 10px; border-bottom: 1px solid #ddd;'>{icon} {item['month']}</td>
                <td style='padding: 10px; border-bottom: 1px solid #ddd;'><strong>{item['action']}</strong></td>
                <td style='padding: 10px; border-bottom: 1px solid #ddd;'>{item['reason']}</td>
                <td style='padding: 10px; border-bottom: 1px solid #ddd; text-align: right;'>{item['prediction']}</td>
            </tr>
            """
        plan_html += "</table>"

        return f"""
        <div class='card'>
            <h4 class='section-title'>📅 12-Month Production Plan</h4>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 1rem; margin-bottom: 1rem;">
                <div>
                    <h5>ML-Generated Recommendations</h5>
                    {plan_html}
                </div>
                <div>
                    {sales_fig.to_html(full_html=False, include_plotlyjs='cdn')}
                </div>
            </div>
        </div>
        """
    except Exception as e:
        return f"<div class='alert-error'>Production plan error: {str(e)}</div>"

# ====================== PREDICTIONS ======================
def predict_customer(age, income, spending_score, purchase_history):
    global models, df
    if not models:
        return "❌ Please train models first", None, None, None, None

    try:
        income_usd = income / 75

        customer_data = pd.DataFrame([[
            age, income_usd, spending_score, purchase_history, 365, purchase_history/365
        ]], columns=['Age', 'Annual_Income', 'Spending_Score', 'Purchase_History',
                    'Days_Since_First_Purchase', 'Purchase_Frequency'])

        # Predict using ML models
        churn_prob = models['churn'].predict_proba(customer_data)[0][1]
        sales_pred = models['sales'].predict(customer_data)[0]
        clv_pred = models['clv'].predict(customer_data)[0]

        # Create consumer profile
        profile_html = f"""
        <div class='consumer-profile'>
            <h4>👤 Target Consumer Profile</h4>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 1rem;">
                <div>
                    <p><strong>Age:</strong> {age} years</p>
                    <p><strong>Income:</strong> ₹{income:,.0f}/year</p>
                </div>
                <div>
                    <p><strong>Spending Score:</strong> {spending_score}/100</p>
                    <p><strong>Purchase History:</strong> {purchase_history} items</p>
                </div>
            </div>
            <p style="margin-top: 1rem;"><i>ML analyzes these attributes to predict customer behavior and value</i></p>
        </div>
        """

        # Create gauges
        churn_gauge = go.Figure(go.Indicator(
            mode="gauge+number",
            value=churn_prob*100,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "<b>Churn Risk %</b><br><i>ML predicts likelihood to stop purchasing</i>", 'font': {'size': 12}},
            gauge={
                'axis': {'range': [None, 100]},
                'steps': [
                    {'range': [0, 30], 'color': "lightgreen"},
                    {'range': [30, 70], 'color': "orange"},
                    {'range': [70, 100], 'color': "red"}],
                'threshold': {
                    'line': {'color': "black", 'width': 4},
                    'thickness': 0.75,
                    'value': churn_prob*100}
            }
        ))
        churn_gauge.update_layout(height=250, margin=dict(t=50, b=10))

        sales_gauge = go.Figure(go.Indicator(
            mode="number",
            value=sales_pred,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "<b>Predicted Purchases</b><br><i>ML forecasts future buying behavior</i>", 'font': {'size': 12}},
            number={'font': {'color': "#3a7bd5"}}
        ))
        sales_gauge.update_layout(height=250, margin=dict(t=50, b=10))

        clv_gauge = go.Figure(go.Indicator(
            mode="number",
            value=clv_pred,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "<b>Predicted CLV (₹)</b><br><i>ML estimates customer lifetime value</i>", 'font': {'size': 12}},
            number={'prefix': "₹", 'valueformat': ",.0f", 'font': {'color': "#1a4b8c"}}
        ))
        clv_gauge.update_layout(height=250, margin=dict(t=50, b=10))

        # Generate recommendation
        recommendation = generate_recommendation(churn_prob, sales_pred, clv_pred)

        return (
            profile_html,
            f"<div class='card'><h4 class='section-title'>📋 ML-Powered Recommendations</h4>{recommendation}</div>",
            churn_gauge,
            sales_gauge,
            clv_gauge
        )
    except Exception as e:
        return f"❌ Prediction error: {str(e)}", None, None, None, None

def generate_recommendation(churn_prob, sales_pred, clv_pred):
    global df
    avg_sales = df['Purchase_History'].mean() if df is not None else 0
    avg_clv = df['Customer_Lifetime_Value'].mean() if df is not None else 0

    # ML-driven business recommendations
    if churn_prob < 0.3:
        churn_status = "🟢 Low Risk"
        action1 = "Maintain engagement"
        detail1 = "ML suggests standard retention strategies"
    elif churn_prob < 0.7:
        churn_status = "🟡 Medium Risk"
        action1 = "Loyalty benefits"
        detail1 = "ML recommends targeted incentives"
    else:
        churn_status = "🔴 High Risk"
        action1 = "Immediate action"
        detail1 = "ML flags for urgent retention efforts"

    if sales_pred > avg_sales * 1.5:
        sales_status = "📈 High Potential"
        action2 = "Upsell premium"
        detail2 = "ML identifies high-value opportunities"
    elif sales_pred > avg_sales:
        sales_status = "📊 Steady"
        action2 = "Cross-sell"
        detail2 = "ML suggests complementary products"
    else:
        sales_status = "📉 Low Activity"
        action2 = "Re-engage"
        detail2 = "ML recommends win-back campaigns"

    if clv_pred > avg_clv * 1.5:
        clv_status = "💰 High Value"
        action3 = "VIP treatment"
        detail3 = "ML identifies top-value customers"
    elif clv_pred > avg_clv:
        clv_status = "💵 Moderate"
        action3 = "Targeted marketing"
        detail3 = "ML suggests personalized offers"
    else:
        clv_status = "💸 Low Value"
        action3 = "Standard service"
        detail3 = "ML recommends cost-efficient approach"

    return f"""
    <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 1rem;">
        <div>
            <h5>Churn Analysis</h5>
            <p><strong>{churn_status}</strong></p>
            <p>Probability: {churn_prob*100:.1f}%</p>
            <p>Action: {action1}</p>
            <p><small>{detail1}</small></p>
        </div>
        <div>
            <h5>Sales Potential</h5>
            <p><strong>{sales_status}</strong></p>
            <p>Predicted: {sales_pred:.1f}</p>
            <p>Action: {action2}</p>
            <p><small>{detail2}</small></p>
        </div>
        <div>
            <h5>Customer Value</h5>
            <p><strong>{clv_status}</strong></p>
            <p>₹{clv_pred:,.0f}</p>
            <p>Action: {action3}</p>
            <p><small>{detail3}</small></p>
        </div>
    </div>
    """

# ====================== GRADIO APP ======================
with gr.Blocks(css=CSS, theme=gr.themes.Default()) as app:
    with gr.Column():
        # Header
        gr.Markdown("""
        <div class='dashboard-header'>
            <h1>Business and Churn Analysis</h1>
            <p>AI-powered customer insights for strategic decisions</p>
        </div>
        """)

        with gr.Tabs():
            # Data & Modeling
            with gr.Tab("📂 Data & Modeling"):
                with gr.Row():
                    with gr.Column(scale=2):
                        gr.Markdown("### 1. Upload and Prepare Data")
                        file_input = gr.File(label="Upload Customer Data (CSV)", file_types=[".csv"])
                        with gr.Row():
                            preprocess_btn = gr.Button("Process Data", variant="primary")
                        preprocess_status = gr.HTML()
                        data_summary = gr.HTML()

                    with gr.Column(scale=3):
                        gr.Markdown("### 2. Train ML Models")
                        with gr.Row():
                            train_btn = gr.Button("Train Models", variant="primary")
                        train_status = gr.HTML()
                        model_cards = gr.HTML()

                        gr.Markdown("### 3. Model Insights")
                        feature_plot = gr.Plot()

            # Analytics
            with gr.Tab("📈 Analytics"):
                sales_analytics = gr.HTML()

            # Predictions
            with gr.Tab("🔮 Predictions"):
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("### Customer Profile")
                        age = gr.Slider(18, 100, value=35, label="Age", step=1)
                        income = gr.Slider(0, 5000000, value=1500000, label="Annual Income (₹)", step=10000)
                        spending_score = gr.Slider(0, 100, value=65, label="Spending Score (0-100)", step=1)
                        purchase_history = gr.Slider(0, 1000, value=120, label="Past Purchases", step=1)
                        predict_btn = gr.Button("Analyze Customer", variant="primary")

                    with gr.Column(scale=2):
                        profile_output = gr.HTML()
                        prediction_output = gr.HTML()
                        with gr.Row():
                            churn_gauge = gr.Plot()
                            sales_gauge = gr.Plot()
                        with gr.Row():
                            clv_gauge = gr.Plot()

            # Planning
            with gr.Tab("📅 Planning"):
                production_plan = gr.HTML()

    # Event handlers
    def update_analytics_and_plan():
        analytics = generate_sales_analytics()
        plan = generate_production_plan()
        return analytics, plan

    preprocess_btn.click(
        preprocess_data,
        inputs=[file_input],
        outputs=[preprocess_status, data_summary]
    ).then(
        update_analytics_and_plan,
        outputs=[sales_analytics, production_plan]
    )

    train_btn.click(
        train_models,
        outputs=[train_status, model_cards, feature_plot]
    )

    predict_btn.click(
        predict_customer,
        inputs=[age, income, spending_score, purchase_history],
        outputs=[profile_output, prediction_output, churn_gauge, sales_gauge, clv_gauge]
    )

app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1cb0b744c84f5e389a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.4 MB/s eta 0:00:00
